# Setting up the Fast Withdrawal contract:
- This notebook allows deploying contracts for fast withdrawal interactions and running a basic scenario by creating a withdrawal on the Etherlink side and making fast payouts on the Tezos side.

In [5]:
from docs.scenarios.setup import *

web3, etherlink_account, tezos_account = setup()

Setup:
- Tezos account: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`, balance: `229.384316 ꜩ`
- Etherlink account: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`, balance: `610.117840863 ꜩ`


## Deploy FastWithdrawal:

In [6]:
FAST_WITHDRAWAL_CONTRACT = 'KT1RweqUvn4sJjCxJuW6BrPpj1AYSywyaNNE'

In [7]:
from scripts.tezos import deploy_fast_withdrawal
from scripts.helpers.contracts.fast_withdrawal import FastWithdrawal

if FAST_WITHDRAWAL_CONTRACT:
    fast_withdrawal = FastWithdrawal.from_address(tezos_account, FAST_WITHDRAWAL_CONTRACT)
    click.echo('Loaded Fast Withdrawal contract, address: ' + wrap(accent(fast_withdrawal.address)))
else:
    fast_withdrawal = deploy_fast_withdrawal.callback(
        xtz_ticketer_address=XTZ_TICKETER_ADDRESS,
        smart_rollup_address=SMART_ROLLUP_ADDRESS,
        tezos_private_key=TEZOS_PRIVATE_KEY,
        tezos_rpc_url=TEZOS_RPC_URL,
        skip_confirm=True,
        silent=False,
    )

Deploying Fast Withdrawal contract:
  - Deployer: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/ghostnet/`
  - Params:
      * Native XTZ Ticketer address: `KT1Bp9YUvUBJgXxf5UrYTM2CGRUPixURqx4m`
      * Smart Rollup address: `sr18wx6ezkeRjt1SZSeZ2UQzQN3Uc3YLMLqg`
Successfully deployed Fast Withdrawal, address: `KT1RweqUvn4sJjCxJuW6BrPpj1AYSywyaNNE`


## Making Fast Withdrawal on Etherlink side:

In [8]:
from scripts.etherlink import xtz_fast_withdraw

tx_hash = xtz_fast_withdraw.callback(
    target=get_address(tezos_account),
    fast_withdrawal_contract=fast_withdrawal.address,
    amount=717 * 10**12,
    discounted_amount=700,
    withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)
tx_hash

Making Fast Withdrawal, XTZ:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Target: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
      * Fast Withdrawal contract: `KT1RweqUvn4sJjCxJuW6BrPpj1AYSywyaNNE`
      * Payload bytes: `0500bc0a`
      * Amount (mutez): `717`
      * Discounted amt (mutez): `700`
      * Fee (mutez): `17`
Successfully initiated XTZ Fast Withdrawal, tx hash: `0xd42021f7898cdf615238edcdce9d296ee07909a46b2086b00be4be311461e3f1`


'0xd42021f7898cdf615238edcdce9d296ee07909a46b2086b00be4be311461e3f1'

### Searching for outbox message:
- TODO: Move this code to a separate script

In [9]:
from scripts.helpers.rollup_node import get_messages
import time
from pytezos import MichelsonType
from pytezos import michelson_to_micheline


def make_event_logs_link(tx_hash):
    return f'https://testnet.explorer.etherlink.com/tx/{tx_hash}?tab=logs'

def make_outbox_message_link(level):
    return f'https://ghostnet-smart.tzkt.io/global/block/{level}/outbox/{level}/messages'

def make_tzkt_ghostnet_link(tx_hash):
    return f'https://ghostnet.tzkt.io/{tx_hash}'

FAST_WITHDRAWAL_OUTBOX_INTERFACE = '''
pair %default (nat %withdrawal_id)
    (pair (ticket %ticket (pair nat (option bytes)))
         (pair (timestamp %timestamp)
               (pair (address %base_withdrawer)
                     (pair (bytes %payload) (bytes %l2_caller)))))
'''

def decode_outbox_message(message):
    micheline_expression = michelson_to_micheline(FAST_WITHDRAWAL_OUTBOX_INTERFACE)
    michelson_type = MichelsonType.match(micheline_expression)
    assert len(message['transactions']) == 1
    parameters_micheline = message['transactions'][0]['parameters']
    return michelson_type.from_micheline_value(parameters_micheline).to_python_object()

def scan_outbox_until_message_found(
    etherlink_rollup_node_url: str,
    last_level: int,
    max_levels: int = 100,
    sleep_time: int = 0.3
):
    click.echo('Scanning outbox: ', nl=False)
    for level in range(last_level, last_level-max_levels, -1):
        click.echo(wrap(accent(level)), nl=False)
        messages = get_messages(ETHERLINK_ROLLUP_NODE_URL, level)
        if len(messages) > 0:
            click.echo()
            click.echo('Found outbox message at level: ' + wrap(accent(level)))
            return messages
        click.echo(', ', nl=False)
        time.sleep(sleep_time)

def make_withdrawal_info(tx_hash, outbox_message):
    click.echo('Fast Withdrawal info:')
    click.echo('- Event logs: ' + make_event_logs_link(tx_hash))
    click.echo('- Outbox message: ' + make_outbox_message_link(tx_hash))
    click.echo('- Withdrawal ID: ' + wrap(accent(outbox_message['withdrawal_id'])))
    click.echo('- Timestamp: ' + wrap(accent(outbox_message['timestamp'])))

In [11]:
last_tezos_level = tezos_account.shell.head()['header']['level']
messages = scan_outbox_until_message_found(ETHERLINK_ROLLUP_NODE_URL, last_tezos_level)
outbox_message = decode_outbox_message(messages[0]['message'])
make_withdrawal_info(tx_hash, outbox_message)

Scanning outbox: `11678307`
Found outbox message at level: `11678307`
Fast Withdrawal info:
- Event logs: https://testnet.explorer.etherlink.com/tx/0xd42021f7898cdf615238edcdce9d296ee07909a46b2086b00be4be311461e3f1?tab=logs
- Outbox message: https://ghostnet-smart.tzkt.io/global/block/0xd42021f7898cdf615238edcdce9d296ee07909a46b2086b00be4be311461e3f1/outbox/0xd42021f7898cdf615238edcdce9d296ee07909a46b2086b00be4be311461e3f1/messages
- Withdrawal ID: `1954`
- Timestamp: `1744051719`


## Purchasing withdrawal on Tezos side:

In [12]:
from scripts.helpers.contracts.fast_withdrawal import Withdrawal
from scripts.helpers.utility import pack
from scripts.helpers.ticket_content import TicketContent

discounted_amount = 700

withdrawal = Withdrawal(
    withdrawal_id=outbox_message['withdrawal_id'],
    full_amount=717,
    ticketer=XTZ_TICKETER_ADDRESS,
    content=TicketContent(
        token_id=0,
        token_info=None,
    ),
    timestamp=outbox_message['timestamp'],
    base_withdrawer=get_address(tezos_account),
    payload=pack(discounted_amount, 'nat'),
    l2_caller=bytes.fromhex(etherlink_account.address.split('0x')[1])
)

opg = fast_withdrawal.payout_withdrawal(
    withdrawal=withdrawal,
    service_provider=get_address(tezos_account),
    xtz_amount=discounted_amount,
).send()

click.echo('- Transaction: ' + make_tzkt_ghostnet_link(opg.hash()))

- Transaction: https://ghostnet.tzkt.io/ooBcqcKtR8oJ9V6j2vbtBojtzsvTABce5GtdmaX4SGkEd7FZLyC
